In [1]:
import requests
import shutil
import os
from urllib import request
from urllib.request import Request, urlopen
import urllib
import html2text as ht
import re
import pandas as pd
import numpy as np
import json
import string
import traceback
from bs4 import BeautifulSoup
from collections import Counter
import glob
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# Scrape Spotify for Top 200 Plays per Week

In [2]:
class DateTracker:
    
    days = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 
            7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    
    def __init__(self, year = None, month = None, day = None):
        self.year = year
        self.month = month
        self.day = day
        
    def previous_week(self):
        
        if (self.year == 1958) and (self.month == 8) and (0 < self.day <= 7):
            return
        
        # if move to previous month
        if self.day - 7 <= 0:
            
            if self.month != 1:
                
                self.month -= 1
                self.day += self.days[self.month] - 7
            
            # if move to previous year
            else:
                
                self.year -= 1
                self.month = 12
                self.day += self.days[self.month] - 7
                
        else:
            
            self.day -= 7
            
    #daniels addition        
    def previous_day(self):
        if (self.year == 1958) and (self.month == 8) and (0 < self.day <= 7):
            return
        if self.day - 1 <= 0: 
            if self.month != 1:
                self.month -= 1
                self.day += self.days[self.month] -1 
            else:
                self.year -= 1
                self.month = 12
                self.day += self.days[self.month] - 1
        else:
            self.day -= 1
        
    def as_date(self):
        
        return "%04d-%02d-%02d" % (self.year, self.month, self.day)

In [3]:
artistData = pd.DataFrame(pd.read_csv("FrostData/artistInfo.csv"))['Artist'].tolist()
allArtists = []
for a in artistData:
    allArtists.append(a.lower())
allArtists.append("khalid")
allArtists.append("billie eilish")

In [4]:
print(len(allArtists))

1165


In [5]:
def scrapeSpotifyPlays(weeks=100):
    path= "https://spotifycharts.com/regional/us/weekly/"#2019-06-07--2019-06-14"
    curdate = DateTracker(year = 2019, month = 7, day = 5)
    datesCsv = []
    for i in range(weeks):
        try:
            endWeek= str(curdate.as_date())
            datesCsv.append(endWeek + ".csv")
            curdate.previous_week()#.as_date()
            req = Request(path + str(curdate.as_date())+"--"+endWeek, headers={'User-Agent': 'Mozilla/5.0'})
            page= urlopen(req).read()
            soup = BeautifulSoup(page, 'html.parser')
            links = soup.find(class_ = "header-csv")
        
            for a in soup.find_all('a', attrs={'class':'header-csv'}, href= True):
                req2= requests.get("https://spotifycharts.com" + a['href'], stream = True)
                if req2.status_code == 200:
                    with open('FrostData/SpotifyWeeks/{0}.csv'.format(endWeek), 'wb') as fx: 
                        req2.raw.decode_content = True
                        shutil.copyfileobj(req2.raw, fx)
                    fx.close()
                    df = pd.read_csv('FrostData/SpotifyWeeks/{0}.csv'.format(endWeek), header = 1)
                    os.remove('FrostData/SpotifyWeeks/{0}.csv'.format(endWeek))
                    df["Week"] = str(endWeek)
                    tempArtists = df["Artist"].values.tolist()
                
                    df.drop('URL', axis= 1, inplace = True)
                    df.to_csv('FrostData/SpotifyWeeks/{0}.csv'.format(endWeek))
        except:
            #There is only data on spotify scraping until 2016
            break
    files = [file for file in glob.glob('FrostData/SpotifyWeeks/*.csv')]
    finalFile = pd.concat([pd.read_csv(file) for file in files])
    returnData = pd.DataFrame(columns = ["Name", "Artist","Features", "Streams", "Week"])
    for index, row in finalFile.iterrows():
        artists = [row["Artist"]]
        try: #Have to handle special case of character difference
            otherArtists = re.findall("\(with (.+)\)",row["Track Name"])
        except:
            continue
        allA = artists
        try:
            allA.extend(otherArtists[0].split(" & "))
        except:
            pass
        inArtist = False
        for artist in allA:
            if artist in allA:
                inArtist =True
                break
        if not inArtist:
            continue
        feat = ""
        if len(otherArtists) != 0:
            trackName = row["Track Name"].split("(", 1)[0]
            otherFeats = otherArtists[0].split(" feat")
            if len(otherFeats) != 1:
                f = otherFeats[1].split(" ", 1)
                feat = f[1]
                otherArtistsJ = ", ".join(otherArtists)
                artists = ", ".join(artists) + ", " + otherArtistsJ.replace(" feat"+otherFeats[1], "")
            else: 
                artists = ", ".join(artists) + ", " + ", ".join(otherArtists)
        else:
            trackName = row["Track Name"]
            featureArtists = re.findall("\(feat(.+)\)", trackName)
            trackName = trackName.split("(feat")[0]
            if len(featureArtists) != 0:
                feat = featureArtists[0].split(" ", 1)[1]
        if type(artists) == list:
            artists = artists[0]
        feat = feat.replace(" & ", ", ")
        artists = artists.replace(" & ", ", ")
        cur = {"Name":trackName ,"Artist": artists,"Features":feat ,"Streams":row["Streams"] , "Week":row["Week"]}
        returnData = returnData.append(cur, ignore_index = True)
    returnData.to_csv('FrostData/spotifyStreams.csv')


In [6]:
#try to get 1043 weeks
scrapeSpotifyPlays(1043)

# Scrape RIAA for label, status,
In 2016, started considering streams in factors to going gold.
Genre was only considered after 2015


In [7]:

#will only need time if the sleep func is needed between click calls
def driverPath(webDriver, xpath):
    link = webDriver.find_element_by_xpath(xpath)
    webDriver.execute_script("arguments[0].click();", link)
    time.sleep(3)
    html = webDriver.page_source
    return html


def scrapeRIAA():#fixed url 
    df = pd.DataFrame(columns = ["Title", "Artist", "Status","Label"])
    url = "https://www.riaa.com/gold-platinum/?tab_active=default-award&ar=&ti=&lab=&genre=&format=Single&date_option=release&from=1999-07-06&to=2019-07-06&award=&type=&category=&adv=SEARCH#search_section"
    #This is all the riaa acolades given to singles released after Jan 1st 2016
    option = webdriver.ChromeOptions()
    option.add_argument('--incognito')
    wd = webdriver.Chrome('/Users/danieldefoe/Frost-2019/chromedriver', options = option)
    wd.get(url)
    html= wd.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    #Continuously push the 'load more results' button to get the driver to the page with all of the listings
    xpath = '//*[@id="loadmore"]'
    prevIter = 0
    while True:
        html = driverPath(wd, xpath)#moves the driver to the next page
        soup = BeautifulSoup(html, "html.parser")
        if len(soup.find_all('tr', attrs = {'class': 'table_award_row'})) > prevIter:
            prevIter = len(soup.find_all('tr', attrs = {'class': 'table_award_row'}))
        else: 
            print('we are looking at all the results')
            break
            
    
    #look at every listing, so every noted single that is certified
    for listing in soup.find_all('tr', attrs = {'class': 'table_award_row'}):
        try:
            artist = listing.find('td', attrs = {'class': 'artists_cell'}).get_text()
        except:
            continue
        #check if it is an artist we are considering -----> use qualifying artists from the billboard 100
        if artist.lower() in allArtists:
            curRow=listing['id']
            #Make xpath to call selenium to expand the "more details" drop down menu
            #htmlNew = driverPath(wd, xpath) #The driver is now on an html page with the extra data we want exposed for the current listing
            #newSoup = BeautifulSoup(htmlNew, "html.parser")#we need a new parser for the new page we are technically on
            title = ''
            try:
                statusText = listing.find('p', attrs = {'class': 'share_text'})['data-share-desc']
                status = re.findall(r" RIAA (.+) Award ", statusText)[0]
            except: 
                #If there is a formatting error due to an apostrophe, consider it gold, as all of these are at least certified gold
                status = "Gold"
            for span in listing.find_all('span'):
                if span.get_text() =="Title:":
                    title = span.nextSibling
                elif span.get_text() == "Label:":
                    label = span.nextSibling
            if title == '':
                val= 1 
                for item in listing.find_all('td', attrs = {'class':'others_cell'}):
                    if val== 1:
                        title = listing.get_text()
                    elif val == 3:
                        label = listing.get_text()
                    val += 1
        
        
            df = df.append({'Title':title, 'Artist':artist, 'Status':status, 'Label':label}, ignore_index= True)
    df.to_csv('FrostData/RIAACertifications.csv')


In [8]:
scrapeRIAA()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  app.launch_new_instance()


we are looking at all the results
 WISH WISH (FEAT CARDI B & 21 SAVAGE)
 LEVEL UP
CON CALMA (FEAT. SNOW)
WISH YOU WERE GAY
EL ANILLO
RUMOR
GOD'S COUNTRY
PUT A DATE ON IT
POWERFUL (FEAT. ELLIE GOULDING & TARRUS RILEY)
STILL FALLING FOR YOU
TRY EVERYTHING
WHAT HURTS THE MOST
SUCKER
UPROAR
NEW LIGHT
CAUGHT UP IN THE COUNTRY
I'M YOURS
DOWN TO THE HONKYTONK
I WAS JACK (YOU WERE DIANE)
HEY LOOK MA, I MADE IT
HIGH HOPES
MONSTER
RIDIN' SOLO
WANT TO WANT ME
WE ARE YOUNG (FEAT. JANELLE MONAE)
HAPPENS LIKE THAT
WHATCHA SAY
WIGGLE (FEAT. SNOOP DOGG)
MIDDLE CHILD
WHAT'S FREE (FEAT. RICK ROSS & JAY Z)
HOPELESS ROMANTIC (FEAT. SWAE LEE)
CHLORINE
I'M THE ONE (FEAT. JUSTIN BIEBER, QUAVO, CHANCE THE RAPPER & LIL WAYNE)
RIGHT WHERE I NEED TO BE
I LOVE COLLEGE
EVERY STORM (RUNS OUT OF RAIN)
NOTHING ON BUT THE RADIO
SONGS ABOUT THE RAIN
BEST I EVER HAD
LIFE AIN'T ALWAYS BEAUTIFUL
SELF CARE
ON MY WAY TO YOU
DO YOU MIND
THROUGH THE FIRE AND FLAMES
KING OF EVERYTHING
BULLETPROOF
STEREO HEARTS
JUMP OUT THE WIN

## Scraping Grammy Winner Data

In [11]:
def scrapeGrammyData(years = 20):
    path = 'https://www.grammy.com/grammys/awards/'
    df = pd.DataFrame(columns = ["Award","GrammyYear", "Genre", "SongTitle", "Artist"])
    for i in range(years):
        year = 61-i
        if year % 10 == 1: 
            suffix = "st"
        elif year % 10 == 2:
            suffix = "nd"
        elif year % 10 == 3:
            suffix = "rd"
        else:
            suffix = "th"
        if year < 21 and year > 3:
            suffix = "th"
        url = path + str(year) + suffix + "-annual-grammy-awards"
        grammyYear = 2018 - i
        req = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
        page= urlopen(req).read()
        soup = BeautifulSoup(page, "html.parser")
        #Currently just individual songs
        validCategories = ["Record Of The Year", "Song Of The Year", "Best Pop Solo Performance", "Best Female Pop Vocal Performance", "Best Male Pop Vocal Performance", "Best Pop Performance By A Duo Or Group With Vocals", "Best Pop Performance By A Duo Or Group With Vocal", "Best Pop Collaboration With Vocals", "Best Female Rock Vocal Performance", "Best Male Rock Vocal Performance", "Best Rock Solo Vocal Performance", "Best Rock Performance By A Duo Or Group With Vocal", "Best Hard Rock Performance", "Best Urban/Alternative Performance", "Best Female R&B Vocal Performance", "Best R&B Performance By A Duo Or Group With Vocals","Best Rap Solo Performance", "Best Rap Performance By A Duo Or Group", "Best Female Rap Solo Performance", "Best Male Rap Solo Performance", "Best Female Country Vocal Performance", "Best Male Country Vocal Performance", "Best Country Performance by a Duo or Group with Vocals", "Best Country Collaboration with Vocals","Best Male R&B Vocal Performance","Best Pop Duo/Group Performance", "Best Dance Recording", "Best Rock Performance", "Best Metal Performance", "Best Rock Song", "Best R&B Performance", "Best Traditional R&B Performance", "Best R&B Song", "Best Rap Performance", "Best Rap/Sung Performance", "Best Rap Song", "Best Country Solo Performance", "Best Country Duo/Group Performance", "Best Country Song", "Best Improvised Jazz Solo", "Best Gospel Performance/Song", "Best Contemporary Christian Music Performance/Song", "Best American Roots Performance", "Best American Roots Song", "Best Song Written For Visual Media"]
        for listing in soup.find_all('div', attrs= {'class': 'view-grouping'}):
            marker = False
            award = listing['data-category']
            if award in validCategories:
                genre = listing['data-genre']
                #Different format for record/performance and song
                if re.search(r'Song', award)!= None and re.search(r'Performance', award)== None:
                    numWinners = len(listing.find_all('span', attrs = {"class": "field-content winner"}))
                    winner = 0
                    for item in listing.find_all('div', attrs = {'class':'wrapper views-fieldset', "data-module":"views_fieldsets"}):
                        #Handle song case
                        if winner < numWinners:
                            winner += 1
                            title = item.find('span', attrs = {'class':'field-content'}).get_text()
                            if year != 58:
                                for text in listing.find_all('p'):
                                    artistText = re.search(r"\(.+\)",text.get_text())
                                    try:
                                        artist = artistText.group()[1:-1]
                                    except AttributeError:
                                        print("This is a special case of the publicly listed artist not having name on the page")
                                        artist =""
                                    break
                            else:
                                for text in listing.find_all('p'):
                                    artistText = re.search(r".+",text.get_text())
                                    artist = artistText.group()[1:-1]
                
                
                else:
                    #Handle performance or record case
                    numWinners = len(listing.find_all('span', attrs = {"class": "field-content winner"}))
                    winner = 0
                    for item in listing.find_all('div', attrs = {'class':'wrapper views-fieldset', "data-module":"views_fieldsets"}):
                        if winner < numWinners:
                            winner += 1 
                            title = item.find('span', attrs = {'class':'field-content'}).get_text()
                            artist = ""
                            for a in item.find_all('a', attrs = {'class':"freelink freelink-nid freelink-internal"}, href = True):
                    
                                if a.nextSibling == None:
                                    artist += str(a.get_text())
                                else:
                                    artist += str(a.get_text())+ str(a.nextSibling)
                            if len(artist.strip()) == 0:
                                for portion in item.find_all('p'):
                                    artist += portion.get_text()
                            if len(artist.strip()) == 0:
                                marker=True
                    if marker:
                        for fs in listing.find_all('fieldset', attrs = {'data-module': 'views_fieldsets'}):
                            for portion in fs.find_all('p'):
                                artist += portion.get_text()
    
                df = df.append({'Award':award, "GrammyYear":grammyYear, "Genre":genre, "SongTitle":title, "Artist":artist}, ignore_index = True)
    #SPECIAL CASE --> best R&B song drunk in love Beyonce ft. JayZ in 57th grammys and other similar formatted listings
    df.to_csv('FrostData/grammyWinners.csv')

In [12]:
scrapeGrammyData()

This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page
This is a special case of the publicly listed artist not having name on the page


# Scrape Chord Data
This assumes that all tabs are in standard tuning.

In [19]:
def transposeToC(driver, key, capo):
    majKeys = ['A', "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]
    transposes = driver.find_elements_by_tag_name("button")
    for t in transposes:
        if t.text== "−1":
            transposeDown= t
        elif t.text == "+1":
            transposeUp = t
   
    #Check flats
    if "b" in key:
        flatKeys = ['A', "Bb", "B", "C", "Db", "D", "Eb", "E", "F", "Gb", "G", "Ab"]
        index = flatKeys.index(key[:2])
        newKey = majKeys[index]
        if "m" in key or "M" in key:
            key = newKey + 'm'
        else:
            key = newKey
            
    #if key is minor, make it its relative major
    if "m" in key or "M" in key:
        oldKey = key.replace('m', "")
        index = majKeys.index(oldKey)
        index = (index + 3)%12
        key = majKeys[index]
    #Now in a major key
    
    if key == 'C':
        return driver
    #account for the capo if it is present, without transposing
    elif capo != 0:
        index = (majKeys.index(key) - capo)
        if index < 0: 
            key = majKeys[12 + index]
        else:
            key = majKeys[index]
    if key == 'C':
        return driver
    elif key in ["F#", "G", "G#", 'A', "A#", "B"]:
        while key != "C":
            index = (majKeys.index(key) + 1)%12
            time.sleep(6)
            transposeUp.click()
            key = majKeys[index]
    elif key in ["C#", "D", "D#", "E", "F"]:
        while key != "C":
            index = (majKeys.index(key) - 1)
            time.sleep(6)
            transposeDown.click()
            key = majKeys[index]
    time.sleep(5)
    return driver
        
    
    
def diatonicPattern(chords):
    cMaj = ["C", "Dm", "Em", "F", "G", "Am", "Bdim"]
    numerals = ["I", "ii", "iii", "IV", "V", "vi", "VII"]
    noExtend=[]
    nonDiatonic = 0
    
    for val in chords:
        if '/' in val:
            if 'm/' in val:
                val = val[0:2]
            else:
                val = val[0]
        noExtend.append(''.join([i for i in val if not i.isdigit()]).replace("maj", "").replace("sus", "").replace('dim', "").replace('aug', "").replace('(', "").replace(')', "").replace('add',""))

    numbers = ""
    last = ""
    prog = []
    for number in noExtend:
        if number != last:
            prog.append(number)
        last = number
    for c in prog:
        try:
            numbers = numbers + "-" + numerals[cMaj.index(c)]
        except:
            cMajStripped = ["C", "D", "E", "F", "G", "A", "B"]
            nonDiatonic += 1
            suffix = ""
            if "#" in c:
                c = c.replace("#", "")
                suffix += "#"
            elif "b" in c:
                c = c.replace('b', "")
                suffix += "b"
            if 'dim' in c:
                c = c.replace('dim', "")
                suffix += ' dim'
            elif 'aug' in c:
                c = c.replace('aug', "")
                suffix += "aug"
            if 'm' in c:
                minor = True
                c = c.replace("m", "")
            elif "M" in c:
                minor = True
                c = c.replace("M", "")
            else:
                minor = False
            #need to handle a major where there should not be one or minor where there should not be one
            if minor: 
                numbers = numbers + "-"+ numerals[cMajStripped.index(c)].lower()+suffix
            else:
                numbers = numbers + "-"+ numerals[cMajStripped.index(c)].upper()+suffix
    return numbers[1:], nonDiatonic

#Function to turn dictionary key value pairs into dataframe rows
def dictToDF(data, song, artist):
    df = pd.DataFrame(columns = ['Name', "Artist", "Section", "Progression", "EndDifferent", "NumSectionChords", "nonDiatonicChords", "extendedChords"])
    for key, value in data.items():
        if value == []:
            continue
        df = df.append({'Name':song, "Artist":artist, "Section":key, "Progression":value[0], "EndDifferent":value[1], "NumSectionChords":value[2], "nonDiatonicChords":value[3], "extendedChords":value[4]}, ignore_index = True)
    return df

In [21]:
def scrapeUltimateGuitar(wd, song, artist):
    url = 'https://www.ultimate-guitar.com/'
    wd.get(url)
    time.sleep(2)
    #Search for the song name 
    searchBarPath = '/html/body/div[1]/div/div/div[2]/div/div/div/p[2]/form/div[2]/input'
    searchBar =  wd.find_element_by_tag_name("input")
    searchBar.send_keys(artist+ " "+song)
    searchBar.send_keys(Keys.ENTER)
    time.sleep(2)
    #Search Chords only 
    try:
        chordsPath = "/html/body/div[1]/div/div/div[2]/main/div[2]/div[3]/section/nav/div[1]/nav/a[2]"
        chords = wd.find_element_by_xpath(chordsPath)
        chords.click()
        time.sleep(2)
    except:
        print("need to wait")
        try:
            time.sleep(20)
            chords = wd.find_element_by_xpath(chordsPath)
            chords.click()
        except:
            print("There was no chords button")
    
    #Pick the top rated listing
    time.sleep(2)
    try:
        bestPath = "/html/body/div[1]/div/div/div[2]/main/div[2]/div[3]/section/nav/div[2]/nav/a[2]"
        highestRated = wd.find_element_by_xpath(bestPath)
        highestRated.click()
        time.sleep(2)
    except:
        print("need to wait")
        try:
            time.sleep(20)
            highestRated = wd.find_element_by_xpath(bestPath)
            highestRated.click()
        except:
            print("ERROR OUT")
    ad = False
    if BeautifulSoup(wd.page_source, 'html.parser').find('title') == "Advertising Page":
        ad =True
        print('ad pops up')
        time.sleep(41)
    curHTML = wd.page_source
    curSoup = BeautifulSoup(curHTML, 'html.parser')
    valid = False
    header = True
    data = None
    for listing in curSoup.find_all('div', attrs = {'class': '_2vnXR'}):
        if header: 
            header = False
            continue
        for form in listing.find_all('div', attrs = {'class':"s2__B _3grw4"}):
            tabType = form.get_text()
        if tabType.lower() == "chords":
            valid = True
            link = listing.find('a', href = True)['href']
            wd.get(link)
            #At this point, we are on the actual tab
            data = chordScraper(wd)
            if data == None:
                wd.execute_script("window.history.go(-1)")
                time.sleep(5)
            else:
                return data
    if not valid:
        print("It does not seem that there is a chord tab for the song " + song + " by " + artist + " on UltimateGuitarTab.com")
        return None
    
    
def chordScraper(wd):#url): #will pass in the driver in real
    html = wd.page_source
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(16)
    #Get key data
    key = ""
    capo = 0
    for headerLine in soup.find_all('div', attrs ={'class':'_17Weg'}):
        if headerLine.get_text()[0:5] == "Capo:":
            capo = int(headerLine.get_text()[6])
        if headerLine.get_text()[0:4] =="Key:":
            key = headerLine.get_text()[5:].strip()
            break
    if key == "":
        #this isn't a valid tab really, we cant standardize it ---> make it go to the next one
        print("NOT A VALID TAB")
        return None
    
    #standardize the key
    transposeToC(wd, key, capo)
    print("NOW WE SHOULD BE IN C MAJOR")
    body = BeautifulSoup(wd.page_source, 'html.parser').find('pre', attrs ={'class':'_3g9-D'})
    #Try to split into different sections
    secs = ["Intro", "Hook","Verse","Chorus","Pre-Chorus","Post-Chorus", "Bridge", "Interlude","Solo","Outro"]
    regex = "\[(.*?)\]"
    sections = re.split(regex,str(body))
    data = {} #Data will be held in a dictionary with the key being the section, and the value being a list of the progression, the end (this one will be None if the end is the same as the whole thing), number of chords in the section, number of nondiatonic chords and number of extended chords 
    secDataNext=False
    secLabel = ""
    #Try to define sections
    for sec in sections:
        #have to take digits away from string to make it see verse 1 and verse 2 as equal for example
        secMod = ''.join([i for i in sec if not i.isdigit()])
        if secMod.strip() in secs:
            #detect if we are actually just finding the label
            secLabel = secMod
            if secLabel not in data.keys():
                data[secLabel] = []
                secDataNext = True
        else:
            miniSoup = BeautifulSoup(sec, 'html.parser')
            secChords= []
            if len(miniSoup.find_all('span', attrs ={'class':'B24oE _1r_2U'})) == 0:
                #This means we are looking at a section of the tab that is not a section label, and does not have any chords
                print('there is nothing useful in this section')
                continue
            for item in miniSoup.find_all('span', attrs ={'class':'B24oE _1r_2U'}):
                chord = item.get_text()
                try:
                    secChords.append(chord)
                except:
                    pass
                
            progression = []
            chords = []
            for c in range(len(secChords)):
                progression.append(secChords[c])
                if c == len(secChords)-1 and chords == []:
                    chords = secChords
                #check if this is the point of circular nature in the progression
                elif len(progression) > 1 and progression[-1] == progression[0]:
                    progLen = len(progression) -1 
                    qualify = True
                    for i in range(progLen):
                        if len(secChords)-1 < progLen + i: 
                            #Then this is the progression
                            chords = secChords
                        #check to see if this pattern repeats itself
                        elif secChords[i] != secChords[progLen + i]:
                            qualify= False
                            break
                    if qualify:
                        chords = progression[:-1]
                        print("Chords are : " + str(chords))
                        break
            if chords == []:
                chords = progression
                        
            #See if the end of the progression is different at all
            end = []
            i = len(chords) -1
            if secChords[-1] == chords[i]:
                if i == len(chords) -1 and i != 0 and secChords[-2] != chords[i-1]: 
                    end = [secChords[-2] , secChords[-1]]
                    counter1 = len(chords) - 3
                    counter2 = -3
                    while secChords[counter2] !=chords[counter1] and counter1 > 0:
                        end = [secChords[counter2]]+ end
                        counter1 -= 1
                        counter2 -= 1
            else:
                end = [secChords[-1]]
                counter1 = len(chords) - 2
                counter2 = -1
                while secChords[counter2] != chords[counter1] and counter1 > 0:
                    end = [secChords[counter2]] + end
                    counter1 -= 1
                    counter2 -= 1
            
            #count extended chords, do not consider the 5 extension, as it is just a part of regular major/minor chord 
            extendedChords = 0
            if end != []:
                for val in end:
                    if not "5" in val and (''.join([i for i in val if i.isdigit()]) != "" or 'sus' in val or 'add' in val):
                        extendedChords += 1
                        val.replace("sus", "")
                    elif '/' in val:
                        extendedChords += 1

            for val in chords:
                if not "5" in val and (''.join([i for i in val if i.isdigit()]) != "" or 'sus' in val or 'add' in val):
                    extendedChords += 1
                    val.replace("sus", "")
                elif '/' in val:
                    extendedChords += 1
            
            #Get number of non-diatonic chords and get the chord pattern in notation
            chords, outKey1 = diatonicPattern(chords)
            
            #do the same for the end pattern
            end, outKey2 = diatonicPattern(end)
            nonDiatonicChords = outKey1 + outKey2
            
            if end == "":
                end = None
            if secDataNext:
                data[secLabel]= [chords, end, len(set(secChords)), nonDiatonicChords, extendedChords]
                secDataNext = False
            if secLabel in list(data.keys()) and data[secLabel] == []:
                print("EMPTY SECTION")
                del data[secLabel]
    
    return data

In [22]:
def scrapeSongChords(archetypeArtists):
    option = webdriver.ChromeOptions()
    option.add_argument('--incognito')
    wd = webdriver.Chrome('/Users/danieldefoe/Frost-2019/chromedriver', options = option)
    billboardSongs = pd.DataFrame(pd.read_csv("FrostData/Weekly_data_clean.csv"))
    done = []
    for index, row in billboardSongs.iterrows():
        if row['Artists'] in archetypeArtists and (row["Name"], row['Artists']) not in done:
            try:
                data = scrapeUltimateGuitar(wd, row["Name"], row['Artists'])
            except:
                print("Diverted from site")
                done.append((row["Name"], row['Artists']))
                continue
            if data == None:
                done.append((row["Name"], row['Artists']))
                continue
            else:
                df = dictToDF(data,row["Name"], row['Artists'])
                done.append((row["Name"], row['Artists']))
                cur = row["Name"] + "-" + row['Artists']
                print(df)
                df.to_csv("FrostData/ArchetypeSongs/{0}.csv".format(cur), encoding='utf-8')
    files = [file for file in glob.glob('FrostData/ArchetypeSongs/*.csv')]
    finalFile = pd.concat([pd.read_csv(file) for file in files])
    finalFile.to_csv('FrostData/songSections.csv')

In [23]:
archetypeArtists =["Maroon 5", "Taylor Swift", "Foo Fighters", "J Cole", "Justin Bieber", "Britney Spears", "Justin Timberlake", "Twenty One Pilots", "Elton John", "Beatles", "Khalid", "Billie Eilish"]
testArtist = ['Khalid']
scrapeSongChords(archetypeArtists)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


NOW WE SHOULD BE IN C MAJOR
Chords are : ['Am', 'Dm', 'E7']
Chords are : ['Am', 'Dm', 'E7']
Chords are : ['Am']
Chords are : ['Am']
      Name         Artist       Section Progression EndDifferent  \
0  Bad Guy  Billie Eilish         Intro   vi-ii-III         None   
1  Bad Guy  Billie Eilish        Verse    vi-ii-III         None   
2  Bad Guy  Billie Eilish       Chorus    vi-ii-III         None   
3  Bad Guy  Billie Eilish  Post-Chorus    vi-ii-III         None   
4  Bad Guy  Billie Eilish     Interlude          vi         None   
5  Bad Guy  Billie Eilish         Outro          vi         None   

  NumSectionChords nonDiatonicChords extendedChords  
0                3                 1              1  
1                3                 1              1  
2                3                 1              1  
3                3                 1              1  
4                1                 0              0  
5                1                 0              0  
NOW WE SHOULD

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"input"}
  (Session info: chrome=75.0.3770.142)
